làm sạch dữ liệu

In [ ]:
import pandas as pd
import datetime as dt
import numpy as numpy

df = pd.read_excel("db.xlsx")

df.info()

Đổi kiểu bảng thành danh sách sản phầm theo hóa đơn

In [ ]:
# Đếm tần suất xuất hiện của mỗi sản phẩm
product_counts = df["Description"].value_counts()

# Lấy 30 sản phẩm xuất hiện nhiều nhất
top_products = product_counts.head(30).index

# Lọc dữ liệu, chỉ giữ lại các sản phẩm trong danh sách 30 sản phẩm phổ biến
filtered_data = df[df["Description"].isin(top_products)]

# Nhóm lại dữ liệu theo InvoiceNo và gộp các sản phẩm
grouped_data = filtered_data.groupby("InvoiceNo")["Description"].apply(list).reset_index()

# Tìm số lượng sản phẩm tối đa trong một hóa đơn sau khi lọc
max_items = grouped_data["Description"].apply(len).max()

# Chuyển dữ liệu hóa đơn thành DataFrame với mỗi sản phẩm là một cột
transformed_data = pd.DataFrame(grouped_data["Description"].tolist(), columns=[f"Item_{i+1}" for i in range(max_items)],index=grouped_data["InvoiceNo"])

# Thay thế các giá trị None bằng NaN
transformed_data = transformed_data.replace([None], np.nan)

# Kết quả
print(f"Số lượng sản phẩm lớn nhất trong một hóa đơn sau khi lọc: {max_items}")
print(transformed_data.head(5))
print(transformed_data.shape)



Dữ liệu sau khi làm sạch

In [ ]:
filtered_transactions = transformed_data[transformed_data.notna().sum(axis=1) >= 2]
print(f"số hóa đơn: {filtered_transactions.shape[0]}")
filtered_transactions.head()

Thông tin chung về mỗi sản phẩm

In [ ]:
# Tạo bảng với các sản phẩm, số lần xuất hiện và tỷ lệ phần trăm
description_counts = transformed_data.apply(pd.Series.value_counts).sum(axis=1).dropna()

# Tính tổng số hóa đơn
total_invoices = transformed_data.shape[0]

# Tạo DataFrame với các cột yêu cầu
summary_table = pd.DataFrame({
    'Description': description_counts.index,   # Tên sản phẩm
    'count': description_counts.values,       # Số lần xuất hiện
    'item_perc': (description_counts.values / total_invoices) * 100  # Tỷ lệ phần trăm
})

# Reset index để Description thành cột
summary_table = summary_table.reset_index(drop=True)

# Sắp xếp bảng theo số lượng sản phẩm giảm dần
summary_table = summary_table.sort_values(by='count', ascending=False)

# In kết quả
print(f"Items count:{total_invoices}")
summary_table.rename(columns={"Description":"item_name"},inplace=True)
summary_table = summary_table.reset_index(drop=True)
summary_table.head()



Biểu đồ

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[16,7])
plt.bar(summary_table["item_name"],summary_table["count"])
plt.ylabel("Item Numbers ->")
plt.xlabel("Item Names ->")
plt.xticks(rotation = 90,fontsize=10)
plt.show()

Xử lí Association Rules:
1. Tạo records để làm tham số cho hàm aptioriaptiori

In [6]:
records = []
row = filtered_transactions.shape[0]
col = filtered_transactions.shape[1]
for i in range(0,row):
    records.append([str(filtered_transactions.values[i,j]) for j in range(0, col)])

tạo association rules

In [7]:
from apyori import apriori
# Tính frequent itemsets
association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
association_results = list(association_rules)


Market Basket Analysis

In [13]:
results = []
for item in association_results:
    pair = item[0]
    items = [x for x in pair]
    
    consequent = str(items[0])
    antecedent = str(items[1])
    support = int(float(str(item[1])[:7]) * 100000)
    confidence = str(item[2][0][2])[:7]
    lift = str(item[2][0][3])[:7]
    
    rows = (consequent,antecedent,support,confidence,lift)
    results.append(rows)
    
    final_result = pd.DataFrame(results,columns=['Consequent','Anticedent','Support','Confidence','Lift'])

In [ ]:
final_result = final_result.sort_values("Support",ascending=False,ignore_index=True)
final_result = final_result[(final_result["Consequent"] != 'nan') & (final_result["Anticedent"] != 'nan')].sort_values("Support",ignore_index=True,ascending=False)
final_result
# for i in range(final_result.shape[0]):
#     print(f"Seems like people who are buying {final_result.Anticedent[i:i+1].values[0]} are more likely to buy {final_result.Consequent[i:i+1].values[0]}.")